<style>
    div.output_scroll {
    height: 1200px;
}
</style>

In [1]:
from dash import Dash, dcc, html, dash_table
import dash_leaflet as dl
from dash.dependencies import Input, Output
import pandas as pd
import matplotlib.pyplot as plt
import urllib.parse

# Import your CRUD module
from AnimalShelter import AnimalShelter


In [2]:
# Connect to MongoDB
username = urllib.parse.quote_plus("aacuser")
password = urllib.parse.quote_plus("Blathras")

shelter = AnimalShelter()

df = pd.DataFrame.from_records(shelter.read({}))

# Clean up the data
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

In [3]:
df.head()


,name,breed,animal_type,age_upon_outcome_in_weeks,location_lat,location_long
0,Lucy,Labrador Retriever,Dog,78,30.75,-97.43
1,Buddy,Labrador Retriever,Dog,52,30.75,-97.48
2,Max,German Shepherd,Dog,80,30.76,-97.47
3,Rex,German Shepherd,Dog,65,30.71,-97.52
4,Luna,Newfoundland,Dog,96,30.78,-97.44


In [ ]:
# Create Dash app layout
app = Dash(__name__)


app.layout = html.Div([
    #html.Img(src='assets/Glogo.png', style={'height': '100px'}),
    html.Center(html.H1("Project Two Dashboard - Steven Blathras")),

    html.H4("Rescue Type Filter"),
    dcc.RadioItems(
        id='rescue-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'All'}
        ],
        value='All',
        labelStyle={'display': 'block'}
    ),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'}
    ),

    html.Br(),
    html.Div(id='map-id', className='col s12 m6'),
    html.Br(),

    dcc.Graph(id='breed-chart'),
    html.Center(html.B("Steven Blathras - SNHU CS-340 MongoDB Dashboard"))
])

In [5]:
@app.callback(
    [Output('datatable-id', 'data'),
     Output('map-id', 'children'),
     Output('breed-chart', 'figure')],
    [Input('rescue-type', 'value'),
     Input('datatable-id', 'selected_rows')]
)
def update_dashboard(rescue_type, selected_rows):
    dog_age_limit = 104  # weeks
    query = {"animal_type": "Dog", "age_upon_outcome_in_weeks": {"$lte": dog_age_limit}}

    rescue_breeds = {
        "Water": ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"],
        "Mountain": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog"],
        "Disaster": ["Doberman Pinscher", "German Shepherd", "Golden Retriever"],
        "All": []
    }

    if rescue_type != "All":
        query["breed"] = {"$in": rescue_breeds[rescue_type]}

    results = shelter.read(query)
    dff = pd.DataFrame.from_records(results)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    # Handle empty dataset
    if dff.empty:
        return [], [html.P("No data available.")], {'data': [], 'layout': {'title': 'No data available'}}

    # Select row index
    row = selected_rows[0] if selected_rows else 0
    if row >= len(dff):
        row = 0

    # Map output
    map_output = [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[
                dff.iloc[row].get("location_lat", 30.75),
                dff.iloc[row].get("location_long", -97.48)],
                children=[
                    dl.Tooltip(dff.iloc[row].get("breed", "Unknown")),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row].get("name", "Unknown"))
                    ])
                ])
        ])
    ]

    # Bar chart of breed counts
    breed_counts = dff['breed'].value_counts()
    chart_fig = {
        'data': [{
            'x': breed_counts.index,
            'y': breed_counts.values,
            'type': 'bar',
            'marker': {'color': 'teal'}
        }],
        'layout': {
            'title': 'Dog Breeds by Count (Filtered)',
            'xaxis': {'title': 'Breed'},
            'yaxis': {'title': 'Count'}
        }
    }

    return dff.to_dict('records'), map_output, chart_fig

In [6]:
# Run dashboard app in jupyter
app.run(debug=True)



[2025-08-18 02:12:14,746] ERROR in app: Exception on /assets/Glogo.png [GET]
Traceback (most recent call last):
  File "C:\Users\jblat\Downloads\CS340_Databases_MongoDashboard(1)\venv\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\jblat\Downloads\CS340_Databases_MongoDashboard(1)\venv\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\jblat\Downloads\CS340_Databases_MongoDashboard(1)\venv\Lib\site-packages\flask\blueprints.py", line 100, in send_static_file
    return send_from_directory(
        t.cast(str, self.static_folder), filename, max_age=max_age
    )
  File "C:\Users\jblat\Downloads\CS340_Databases_MongoDashboard(1)\venv\Lib\site-packages\flask\helpers.py", line 565, in send_from_directory
    re